In [1]:
from dgl.data.utils import save_graphs
import dgl
import torch as th
import scipy.sparse as sp
import networkx as nx
import pickle
from numpy import array
import pandas as pd

#Aprire i File
import os
#Per la gestione dei file già esistenti
from os.path import exists

Using backend: pytorch


In [11]:
proteine_path = "../Dati/proteine.csv"
proteine = pd.read_csv(proteine_path, sep = ',')

f = open("../Dati/classi.pickle","rb")
classi = pickle.load(f)
f.close()

f = open("../Dati/aminoacidi.pickle","rb")
amino = pickle.load(f)
f.close()


for filename in os.listdir("../../R/Rdati/Matrici Adiacenza/"):
    with open(os.path.join("../../R/Rdati/Matrici Adiacenza/", filename), 'r', encoding='windows-1252') as f:
        if (filename != ".DS_Store"):
            id_p = filename[0:4]
            #print(id_p)

            #Id della proteina che si sta analizzando
            #print(id_p)

            if not exists("../Dati/Grafi2/{}.npz".format(id_p)): 
                matrix = "../../R/Rdati/Matrici Adiacenza/{}.csv".format(id_p)
                #print(matrix)

                #Matrice adiacenza singola matrice inserita in un dataframe
                proteina = pd.read_csv(matrix, sep = ';')
                #Nomi dei nodi nonché numero
                Nodes = list(proteina.columns[1:])
                for vertex in range(0,len(Nodes)):
                    Nodes[vertex] = Nodes[vertex][1:]
                    
                #Lista degli Amino acidi
                    #Amino = list(proteina.iloc[0][1:])
                Amino = amino.transform(list(proteina.iloc[0][1:]))

                
                proteina.drop(proteina.columns[[0]], axis = 1, inplace = True)
                j = proteina.iloc[0]
                proteina.drop([0], axis = 0, inplace = True)
                
                
                    #proteina.drop(proteina.columns[[0]], axis = 1, inplace = True)
                    #proteina.drop([0], axis = 0, inplace = True)
                #Classe della proteina
                classe = proteine[proteine['Id'] == id_p]["Classe"].iloc[0][0]
        
                
                #Creazione Grafo
                G = nx.Graph()
                G.add_nodes_from(Nodes)
                
                #Aggiunta archi
                rows, cols = proteina.shape
                Vprimo = "Vx"
                Vsecondo ="Vx"
                for i in range(rows):
                    row = proteina.iloc[i]
                    j = 0
                    for edge in row:
                        if edge == '1':
                            vPrimo = Vprimo[:1] + str(i+1)
                            vSecondo = Vsecondo[:1] + str(j+1)
                            #print("Aggiunta arco tra il nodo ", vPrimo[1:],"e il nodo ", vSecondo[1:])
                            G.add_edge(vPrimo[1:], vSecondo[1:])
                        j += 1
                
                #Conbersione Grafo in formato DGL
                new_g = dgl.from_networkx(G)
                
                
                #Salvataggio Grafo
                save_graphs("../Dati/Grafi2/{}.bin".format(id_p), new_g)
                break
        

In [10]:
id_p

'4vhb'

In [9]:
new_g

Graph(num_nodes=138, num_edges=4174,
      ndata_schemes={}
      edata_schemes={})